In [1]:
import random
import time

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import torch

from src.model.activation import Relu, LeakyRelu, Linear, Sigmoid
from src.model.base import NeuralNetwork
from src.model.layer import Convolutional, Input3D, Input, FullyConnected, Flatten
from src.model.loss import CategoricalCrossEntropy
from src.model.metric import AccuracyOneHot
from src.model.optimizer import Adam, SGD

In [2]:
# train_data = [
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [
#             [
#                 [1.0, 0.0, 1.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 1.0, 1.0, 0.0],
#             ]
#         ],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [
#             [
#                 [0.0, 1.0, 0.0, 0.0, 1.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [0.0, 0.0, 1.0, 0.0, 0.0],
#                 [0.0, 1.0, 0.0, 1.0, 0.0],
#                 [1.0, 0.0, 0.0, 0.0, 1.0],
#             ]
#         ],
#         'output': [0.0, 1.0]
#     }
    
# ]

In [3]:
# layers = [
#     Convolutional(LeakyRelu(), filters_num=16, kernel_size=2),
#     Convolutional(LeakyRelu(), filters_num=16, kernel_size=2),
#     Flatten(),
#     #FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input3D((1, 5, 5)),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )


# start_time = time.time()
# epochs = 100
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_111')
# import sys
# sys.exit(0)

In [4]:
def get_onehot_digit(digit):
    output = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    return output[-digit:] + output[:-digit]

In [5]:
def get_digits_data():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": input_values,
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:400], train_dataset[400:500]

train_dataset, test_dataset = get_digits_data()

layers = [
    FullyConnected(256, LeakyRelu()),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(64, LeakyRelu()),
    FullyConnected(10, Linear()),
]
nn = NeuralNetwork(
    Input(784),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction="argmax",
    on_cuda=False
)

start_time = time.time()

epochs = 10
history = nn.fit(train_dataset, test_dataset, epochs=epochs, batch_size=1, verbose=True)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 1/10, train loss: 2.05, train Accuracy: 0.55, test loss: 1.9547, test Accuracy: 0.64, epoch time: 0.605s
Epoch: 2/10, train loss: 1.7897, train Accuracy: 0.83, test loss: 1.7943, test Accuracy: 0.74, epoch time: 0.578s
Epoch: 3/10, train loss: 1.6587, train Accuracy: 0.91, test loss: 1.6958, test Accuracy: 0.83, epoch time: 0.546s
Epoch: 4/10, train loss: 1.5767, train Accuracy: 0.9725, test loss: 1.7187, test Accuracy: 0.82, epoch time: 0.584s
Epoch: 5/10, train loss: 1.5404, train Accuracy: 0.9775, test loss: 1.7236, test Accuracy: 0.79, epoch time: 0.563s
Epoch: 6/10, train loss: 1.5183, train Accuracy: 0.99, test loss: 1.8062, test Accuracy: 0.74, epoch time: 0.567s
Epoch: 7/10, train loss: 1.5053, train Accuracy: 0.995, test loss: 1.6864, test Accuracy: 0.8, epoch time: 0.547s
Epoch: 8/10, train loss: 1.4905, train Accuracy: 0.9975, test loss: 1.6913, test Accuracy: 0.81, epoch time: 0.554s
Epoch: 9/10, train loss: 1.4834, train Accuracy: 1.0, test loss: 1.698, test Accurac

In [6]:
def plot_digit(image):
    image = torch.tensor(image).numpy().reshape(28, 28)
    plt.figure()
    plt.imshow(image, cmap="gray")
    plt.axis("off")
    plt.show()

    return

In [7]:
def get_digits_data_cnn():

    train_dataset = []

    train = pd.read_csv("../data/digits/train.csv", header=0, nrows=5000)

    for index in train.index:
        input_values = [
            float(val) / 255.0 for val in train.loc[index].drop("label").values
        ]
        train_dataset.append(
            {
                "input": [torch.tensor(input_values).reshape(28, 28).tolist()],
                "output": get_onehot_digit(int(train.loc[index]["label"])),
            }
        )

    # print(train_dataset[:2])
    # import sys
    # sys.exit(1)
    
    random.seed(1)
    random.shuffle(train_dataset)
    return train_dataset[:400], train_dataset[400:500]

In [8]:
train_data, test_data = get_digits_data_cnn()

layers = [
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    Convolutional(LeakyRelu(), filters_num=4, kernel_size=3),
    #Convolutional3x3x16x0x1(LeakyRelu(), kernel=2),
    Flatten(),
    FullyConnected(128, LeakyRelu()),
    FullyConnected(10, Linear())
]
nn = NeuralNetwork(
    Input3D((1, 28, 28)),
    layers,
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossEntropy(),
    metric=AccuracyOneHot(),
    convert_prediction='argmax',
    on_cuda=False
)

#plot_digit(train_data[0]['input'][0])

start_time = time.time()
epochs = 10
nn.fit(train_data, test_data, epochs=epochs, verbose=True)
print('exec time: ', time.time() - start_time)

print('test_111')

Epoch: 1/10, train loss: 2.0914, train Accuracy: 0.5375, test loss: 1.8123, test Accuracy: 0.71, epoch time: 185.906s
Epoch: 2/10, train loss: 1.785, train Accuracy: 0.89, test loss: 1.8888, test Accuracy: 0.8, epoch time: 186.249s
Epoch: 3/10, train loss: 1.6642, train Accuracy: 0.9675, test loss: 1.7794, test Accuracy: 0.86, epoch time: 185.986s
Epoch: 4/10, train loss: 1.5888, train Accuracy: 0.995, test loss: 1.7791, test Accuracy: 0.86, epoch time: 182.838s
Epoch: 5/10, train loss: 1.5702, train Accuracy: 0.9975, test loss: 1.8263, test Accuracy: 0.83, epoch time: 187.625s
Epoch: 6/10, train loss: 1.5373, train Accuracy: 0.9975, test loss: 1.8259, test Accuracy: 0.82, epoch time: 200.488s
Epoch: 7/10, train loss: 1.52, train Accuracy: 1.0, test loss: 1.7441, test Accuracy: 0.86, epoch time: 202.067s
Epoch: 8/10, train loss: 1.501, train Accuracy: 1.0, test loss: 1.7986, test Accuracy: 0.86, epoch time: 200.33s
Epoch: 9/10, train loss: 1.502, train Accuracy: 1.0, test loss: 1.829, 

In [9]:
# train_data = [
#     {
#         'input': [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# test_data = [
#     {
#         'input': [1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#         'output': [1.0, 0.0]
#     },
#     {
#         'input': [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
#         'output': [0.0, 1.0]
#     }    
# ]

# layers = [
#     FullyConnected(25, LeakyRelu()),
#     FullyConnected(10, LeakyRelu()),
#     FullyConnected(2, Linear())
# ]
# nn = NeuralNetwork(
#     Input(25),
#     layers,
#     optimizer=SGD(),
#     loss=CategoricalCrossEntropy(),
#     metric=AccuracyOneHot(),
#     convert_prediction='argmax'
# )

# start_time = time.time()
# epochs = 200
# nn.fit(train_data, test_data, epochs=epochs, verbose=True)
# print('exec time: ', time.time() - start_time)

# print('test_222')